#### This file is used to analyse the relationship between simulation number and the standard error from the analytical solution

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import glob
import numpy as np
from scipy.stats import norm


## get a plot of 
# standard error vs N vs phase space params of the blackscholes hmm

# then apply it to asian options hmm

In [2]:
data_dir = Path("../data")
csv_files = list(data_dir.glob("*.csv"))
csv_files

[PosixPath('../data/BlackScholes_S100_K100_r0.05_sigma0.2_T1_steps252_paths100000.csv'),
 PosixPath('../data/BlackScholes_S100_K100_r0.05_sigma0.2_T1_steps252_paths1000.csv'),
 PosixPath('../data/BlackScholes_S100_K100_r0.05_sigma0.2_T1_steps252_paths10.csv')]

In [7]:
csv_files[0]

PosixPath('../data/BlackScholes_S100_K100_r0.05_sigma0.2_T1_steps252_paths100000.csv')

In [ ]:
# Define correct Black-Scholes analytical solution
def black_scholes_analytical(S, K, T, r, sigma, t):
    """Calculate Black-Scholes analytical solution for stock price evolution.
    This gives the expected value of the stock price at time t."""
    # Expected value of geometric Brownian motion
    return S * np.exp(r * t)  # This is the expected value under risk-neutral measure

latest_file = max(csv_files, key=lambda x: x.stat().st_mtime)
print(f"Loading file: {latest_file.name}")

# Extract parameters from filename
params = latest_file.stem.split('_')
model_name = params[0]

# Extract all parameters including sigma - now handling decimal points directly
params_dict = {}
for p in params[1:]:
    if p.startswith('S'):
        params_dict['S'] = float(p[1:])  # Just convert directly
    elif p.startswith('K'):
        params_dict['K'] = float(p[1:])
    elif p.startswith('r'):
        params_dict['r'] = float(p[1:])
    elif p.startswith('T'):
        params_dict['T'] = float(p[1:])
    elif p.startswith('sigma'):
        params_dict['sigma'] = float(p[5:])  # Remove 'sigma' prefix and convert

print("Extracted parameters:", params_dict)

# Read the CSV file
df = pd.read_csv(latest_file)
print(f"Data shape: {df.shape}")

# Calculate analytical solution
times = df['time'].values
analytical_prices = black_scholes_analytical(
    params_dict['S'],
    params_dict['K'],
    params_dict['T'],
    params_dict['r'],
    params_dict['sigma'],
    times
)

# Create the plot
plt.figure(figsize=(12, 8))

# Plot individual paths with low opacity
path_columns = [col for col in df.columns if col.startswith('path_')]
for col in path_columns:
    plt.plot(df['time'], df[col], 'b-', alpha=0.2, linewidth=1)

# Plot average path with higher opacity and thickness
plt.plot(df['time'], df['average_path'], 'r-',
            linewidth=2, label='Average Path')

# Add analytical solution
plt.plot(times, analytical_prices, 'k--',
         linewidth=2, label='Analytical Solution', alpha=0.7)


# Add horizontal line at strike price if available
if 'K' in params_dict:
    plt.axhline(y=params_dict['K'], color='g', linestyle='--',
                alpha=0.5, label=f"Strike Price (K={params_dict['K']})")

# Customize the plot
plt.title(f"{model_name} Stock Price Paths\n" +
            f"S₀={params_dict['S']}, r={params_dict['r']}, T={params_dict['T']}")
plt.xlabel("Time (years)")
plt.ylabel("Stock Price")
plt.grid(True, alpha=0.3)
plt.legend()

# Add text box with parameters
param_text = "\n".join([
    f"{k}={v:.4f}" for k, v in params_dict.items()
])
plt.text(0.02, 0.98, param_text,
            transform=plt.gca().transAxes,
            verticalalignment='top',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

plt.show()

# Print some statistics
print("\nPath Statistics:")
print(f"Path Name: {latest_file}")
print(f"Initial price: {df.iloc[0]['average_path']:.4f}")
print(f"Final average price: {df.iloc[-1]['average_path']:.4f}")
print(f"Final analytical price: {analytical_prices[-1]:.4f}")
print(f"Price range: [{df.iloc[-1][path_columns].min():.4f}, {df.iloc[-1][path_columns].max():.4f}]")
print(f"Monte Carlo vs Analytical difference: {abs(df.iloc[-1]['average_path'] - analytical_prices[-1]):.4f}")


Data shape: (253, 12)


,time,path_0,path_1,path_2,path_3,path_4,path_5,path_6,path_7,path_8,path_9,average_path
0,0.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
1,0.003968,99.217135,99.975629,99.415157,99.342224,97.859126,99.881639,100.729945,99.728130,100.824761,100.648674,100.020942
2,0.007937,98.957163,99.807787,97.318356,99.821318,97.576781,100.935256,98.287260,103.041911,98.117867,101.554142,100.046212
3,0.011905,100.307193,100.421078,97.247205,99.317710,97.092499,102.260590,99.455400,104.480634,97.895380,102.261290,100.052561
4,0.015873,101.895456,99.662747,96.301887,102.431071,96.346333,103.591212,100.161640,102.945875,97.272864,101.910415,100.071098
